Opis:

funkcją rozwiązującą zadanie jest send_binary_message

przyjmuje ona stringa którego chcemy nadać oraz rozmiar paczki, czyli kawałków na które go dzielimy

i dla każdego z tych kawałków pakuję w ramkę i ślę

In [3]:
import numpy as np
import pulseaudio as pa
import matplotlib.pyplot as plt
import time as ti
import binascii
import uuid
import zlib
import sys
import termios
import tty
import time
from bitarray import bitarray
from Encoder import Encoder
%matplotlib inline

class Sender(object):
    def __init__(self, frequencies=[1764, 4410], amplitude=2000, bit_length=100):
        self.amplitude = amplitude
        self.frequencies = frequencies
        self.framerate = 44100
        self.points = [self.framerate / self.frequencies[x] for x in [0, 1]]
        self.values = [np.linspace(start=0, stop=2*np.pi, num=self.points[x], endpoint=False) for x in [0, 1]]
        self.values = [list(self.values[x]) * (bit_length/self.points[x]) for x in [0, 1]]
        self.values = [np.round(self.amplitude * np.sin(self.values[x])) for x in [0, 1]]
        self.tones = [(self.values[x]).astype(np.int16).tostring() for x in [0, 1]]
        self.mac = ':'.join(("%012X" % uuid.getnode())[i:(i+2)] for i in range(0, 12, 2))
    
    def generate_tone_time(self, bit, time_l):
        tune = self.tones[bit]
        current_time = time.time()
        end_time = current_time + time_l
        with pa.simple.open(direction=pa.STREAM_PLAYBACK, 
                            format=pa.SAMPLE_S16LE, 
                            rate=self.framerate, 
                            channels=1) as player:
            while(time.time() < end_time):
                player.write(tune)        
        return
    
    def send_binary_string(self, s):
        """
            wysyła string s (bez opakowywania)
        """
        message = ''.join([self.tones[0 if b=="0" else 1] for b in s])
        with pa.simple.open(direction=pa.STREAM_PLAYBACK, 
                            format=pa.SAMPLE_S16LE, 
                            rate=self.framerate, 
                            channels=1) as player:
            player.write(message)
        return
    
    def send_binary_message(self, s, package_size=1499):
        """
            dzieli stringa s na paczki, opakowuje je w ramki i wysyła
            package_size musi dawać resztę 3 z dzielenia przez 4
        """
        frames = Encoder.get_frames(self.mac, self.mac, s, package_size)
        for ramka in frames:
            self.send_binary_string(ramka)
            time.sleep(0.3) #to musi być, bo są opóźnienia związanie z obróbką wiadomości

Importing ipynb file : Encoder.ipynb


In [11]:
s = Sender(bit_length=50)

stach = ("Witajcie! "
"Nazywam sie Jacek Laszczok Stachursky "
"Urodzilem sie dwudziestego szostego stycznia tysiac dziewiecset szescdziesiatego szostego roku "
"Jestem Zolta Magnetyczna Gwiazda, ktora otacza centralny, zielony Zamek Synchronizacji "
"Przewodnikiem mi Zolty Samoistny Czlowiek "
"Jestem otwarty dla wyzszych wymiarow "
"I przekraczam granice wyobrazni! "
"Tworze piekno i harmonie gdziekolwiek sie pojawie "
"Po mojej prawicy zasiada czerwony, Galaktyczny Wedrowiec "
"Po mojej lewicy - niebieska, wiodaca Moc "
"Przez zycie prowadzi mnie biale, Rytmiczne Zwierciadlo... "
"Witam Was w moim swiecie! "
"Bawcie sie i cieszcie razem ze mna... "
"Jam jest czterysta czterdziesci... i cztery.")

st = time.time()
s.send_binary_message(Encoder.string_to_bits(stach), package_size=1999)
et = time.time()
print et-st

11.3374378681
